# Document Intelligence 로 컨텐츠 추출하기
 - 이 노트북에서는 Azure Document Intelligence를 사용하여 PDF, PPTX 등 다양한 문서에서 
 - 텍스트와 구조 정보를 추출하는 방법을 실습합니다.

## 📋 목차
- 1. 환경 설정 및 라이브러리 설치
- 2. Azure Document Intelligence 클라이언트 설정
- 3. PDF 문서 분석 (Layout 모델)
- 4. PPTX 문서 분석 (Layout 모델)
- 5. 분석 결과를 마크다운으로 저장

## 1. 환경 설정 및 라이브러리 설치

Azure Document Intelligence SDK와 필요한 라이브러리를 설치합니다.

In [1]:
# 필요한 패키지 설치
# azure-ai-documentintelligence: Document Intelligence SDK
# azure-identity: Azure 인증을 위한 SDK
# python-dotenv: 환경 변수 로드

%pip install azure-ai-documentintelligence azure-identity python-dotenv

Note: you may need to restart the kernel to use updated packages.


## 2. Azure Document Intelligence 클라이언트 설정

환경 변수에서 Azure Document Intelligence 엔드포인트를 로드하고 클라이언트를 초기화합니다.

### 인증 방식 선택
두 가지 인증 방식 중 선택할 수 있습니다:

| 방식 | 설명 | 환경 변수 |
|------|------|----------|
| **API 키** | 간단한 키 기반 인증 (권장: 로컬 개발) | `AZURE_DOCUMENT_INTELLIGENCE_KEY` |
| **DefaultAzureCredential** | Azure AD 기반 인증 (`az login` 필요) | - |

### 사전 준비사항
`.env` 파일에 다음 환경 변수를 설정하세요:
```
AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT=https://your-resource.cognitiveservices.azure.com/
AZURE_DOCUMENT_INTELLIGENCE_KEY=your-api-key-here
```

In [2]:
# 라이브러리 임포트
import os
from dotenv import load_dotenv
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult
from azure.ai.documentintelligence.models import DocumentContentFormat

# .env 파일에서 환경 변수 로드
load_dotenv()

# Azure Document Intelligence 엔드포인트 설정
endpoint = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")

# API 키 가져오기 (키 인증 시 사용)
api_key = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_KEY")

print(f"📍 Endpoint: {endpoint}")
print(f"🔑 API Key: {'설정됨' if api_key else '설정 안됨'}")

📍 Endpoint: https://azure-ai-services-1023.cognitiveservices.azure.com/
🔑 API Key: 설정됨


### 방법 1: API 키 인증 (az login 불필요)

API 키를 사용하여 인증합니다. `az login` 없이 바로 사용할 수 있습니다.

In [3]:
# 방법 1: API 키 인증
# AzureKeyCredential을 사용하여 API 키로 인증
from azure.core.credentials import AzureKeyCredential

# API 키가 설정되어 있는지 확인
if api_key:
    # API 키로 클라이언트 생성
    credential = AzureKeyCredential(api_key)
    document_intelligence_client = DocumentIntelligenceClient(
        endpoint=endpoint, 
        credential=credential
    )
    print("✅ API 키 인증으로 Document Intelligence 클라이언트 초기화 완료")
else:
    print("❌ API 키가 설정되지 않았습니다. .env 파일에 AZURE_DOCUMENT_INTELLIGENCE_KEY를 설정하세요.")

✅ API 키 인증으로 Document Intelligence 클라이언트 초기화 완료


### 방법 2: DefaultAzureCredential 인증 (az login 필요)

Azure CLI로 로그인한 계정을 사용하여 인증합니다. 프로덕션 환경에서 권장됩니다.

In [4]:
# 방법 2: DefaultAzureCredential 인증 (az login 필요)
# Azure CLI, 환경 변수, Managed Identity 등 다양한 인증 방식을 자동으로 시도
from azure.identity import DefaultAzureCredential

# DefaultAzureCredential 사용 시 아래 코드 실행
# 주의: 이 방법을 사용하려면 먼저 터미널에서 `az login` 실행 필요

# credential = DefaultAzureCredential()
# document_intelligence_client = DocumentIntelligenceClient(
#     endpoint=endpoint, 
#     credential=credential
# )
# print("✅ DefaultAzureCredential로 Document Intelligence 클라이언트 초기화 완료")

print("ℹ️  DefaultAzureCredential 방식을 사용하려면 위 코드의 주석을 해제하세요.")
print("    사전에 터미널에서 `az login` 명령으로 Azure에 로그인해야 합니다.")

ℹ️  DefaultAzureCredential 방식을 사용하려면 위 코드의 주석을 해제하세요.
    사전에 터미널에서 `az login` 명령으로 Azure에 로그인해야 합니다.


## 3. 샘플 파일 확인

`tutorials/samples` 폴더에 있는 샘플 파일들을 확인합니다.
- **PDF**: Accelerating-Sustainability-with-AI-2025.pdf
- **PPTX**: Cognitive Searvices and Content Intelligence.pptx

In [5]:
# 샘플 파일 경로 설정
samples_dir = "samples"

# PDF 샘플 파일
pdf_file_path = os.path.join(samples_dir, "Accelerating-Sustainability-with-AI-2025.pdf")

# PPTX 샘플 파일
pptx_file_path = os.path.join(samples_dir, "Cognitive Searvices and Content Intelligence.pptx")

# 파일 존재 여부 확인
print("📂 샘플 파일 확인:")
print("-" * 60)

for file_path in [pdf_file_path, pptx_file_path]:
    if os.path.exists(file_path):
        file_size = os.path.getsize(file_path) / 1024  # KB 단위
        print(f"✅ {os.path.basename(file_path)} ({file_size:.1f} KB)")
    else:
        print(f"❌ {file_path} - 파일을 찾을 수 없습니다.")

📂 샘플 파일 확인:
------------------------------------------------------------
✅ Accelerating-Sustainability-with-AI-2025.pdf (6818.3 KB)
✅ Cognitive Searvices and Content Intelligence.pptx (11230.2 KB)


## 4. PDF 문서 분석 (Layout 모델)

Layout 모델을 사용하여 PDF 문서에서 텍스트, 테이블, 구조 정보를 추출합니다.

### Layout 모델의 특징
- 텍스트 추출 (OCR)
- 테이블 구조 인식
- 체크박스/선택 마크 인식
- 문서 레이아웃 분석
- 마크다운 형식으로 출력 가능

In [6]:
def analyze_pdf_to_markdown(file_path: str) -> AnalyzeResult:
    """
    PDF 파일을 분석하여 마크다운 형식으로 컨텐츠를 추출합니다.
    
    Args:
        file_path: PDF 파일 경로
        
    Returns:
        AnalyzeResult: 분석 결과 객체
    """
    # 파일 존재 여부 확인
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"파일을 찾을 수 없습니다: {file_path}")
    
    print(f"📂 PDF 분석 시작: {file_path}")
    print("-" * 60)
    
    # 로컬 파일을 바이너리로 읽기
    with open(file_path, "rb") as f:
        file_content = f.read()
    
    # Layout 모델로 PDF 분석 시작
    # output_content_format을 MARKDOWN으로 설정하여 마크다운 형식으로 결과 출력
    poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-layout",  # Layout 모델 사용
        body=file_content,
        content_type="application/pdf",  # PDF 컨텐츠 타입
        output_content_format=DocumentContentFormat.MARKDOWN  # 마크다운 형식으로 출력
    )
    
    # 분석 완료 대기 및 결과 반환
    result: AnalyzeResult = poller.result()
    
    print(f"✅ PDF 분석 완료!")
    return result

In [7]:
# PDF 분석 실행
pdf_result = analyze_pdf_to_markdown(pdf_file_path)

📂 PDF 분석 시작: samples/Accelerating-Sustainability-with-AI-2025.pdf
------------------------------------------------------------
✅ PDF 분석 완료!


In [8]:
def print_analysis_summary(result: AnalyzeResult, doc_type: str = "문서"):
    """
    분석 결과 요약을 출력합니다.
    
    Args:
        result: Document Intelligence 분석 결과
        doc_type: 문서 타입 (PDF, PPTX 등)
    """
    print(f"\n📊 {doc_type} 분석 결과 요약")
    print("=" * 60)
    
    # 페이지 정보
    if result.pages:
        print(f"📄 총 페이지 수: {len(result.pages)}")
        for page in result.pages:
            print(f"   - 페이지 {page.page_number}: {page.width} x {page.height} {page.unit}")
    
    # 테이블 정보
    if result.tables:
        print(f"\n📋 테이블 수: {len(result.tables)}")
        for idx, table in enumerate(result.tables):
            print(f"   - 테이블 {idx + 1}: {table.row_count}행 x {table.column_count}열")
    
    # 단락 정보
    if result.paragraphs:
        print(f"\n📝 단락 수: {len(result.paragraphs)}")
    
    # 그림/이미지 정보
    if result.figures:
        print(f"\n🖼️  그림/이미지 수: {len(result.figures)}")
        for idx, figure in enumerate(result.figures):
            page_num = figure.bounding_regions[0].page_number if figure.bounding_regions else "N/A"
            caption = ""
            if hasattr(figure, 'caption') and figure.caption:
                caption = f" - 캡션: {figure.caption.content}"
            print(f"   - 그림 {idx + 1}: 페이지 {page_num}{caption}")
    
    print("=" * 60)

# PDF 분석 결과 요약 출력
print_analysis_summary(pdf_result, "PDF")


📊 PDF 분석 결과 요약
📄 총 페이지 수: 36
   - 페이지 1: 8.5 x 11.0 LengthUnit.INCH
   - 페이지 2: 8.5 x 11.0 LengthUnit.INCH
   - 페이지 3: 8.5 x 11.0 LengthUnit.INCH
   - 페이지 4: 8.5 x 11.0 LengthUnit.INCH
   - 페이지 5: 8.5 x 11.0 LengthUnit.INCH
   - 페이지 6: 8.5 x 11.0 LengthUnit.INCH
   - 페이지 7: 8.5 x 11.0 LengthUnit.INCH
   - 페이지 8: 8.5 x 11.0 LengthUnit.INCH
   - 페이지 9: 8.5 x 11.0 LengthUnit.INCH
   - 페이지 10: 8.5 x 11.0 LengthUnit.INCH
   - 페이지 11: 8.5 x 11.0 LengthUnit.INCH
   - 페이지 12: 8.5 x 11.0 LengthUnit.INCH
   - 페이지 13: 8.5 x 11.0 LengthUnit.INCH
   - 페이지 14: 8.5 x 11.0 LengthUnit.INCH
   - 페이지 15: 8.5 x 11.0 LengthUnit.INCH
   - 페이지 16: 8.5 x 11.0 LengthUnit.INCH
   - 페이지 17: 8.5 x 11.0 LengthUnit.INCH
   - 페이지 18: 8.5 x 11.0 LengthUnit.INCH
   - 페이지 19: 8.5 x 11.0 LengthUnit.INCH
   - 페이지 20: 8.5 x 11.0 LengthUnit.INCH
   - 페이지 21: 8.5 x 11.0 LengthUnit.INCH
   - 페이지 22: 8.5 x 11.0 LengthUnit.INCH
   - 페이지 23: 8.5 x 11.0 LengthUnit.INCH
   - 페이지 24: 8.5 x 11.0 LengthUnit.INCH
   - 페이지 25: 8.5 x 

In [9]:
# 추출된 마크다운 콘텐츠 미리보기 (처음 2000자)
print("📄 추출된 마크다운 콘텐츠 (미리보기)")
print("=" * 60)

if pdf_result.content:
    preview_length = 2000
    content_preview = pdf_result.content[:preview_length]
    print(content_preview)
    if len(pdf_result.content) > preview_length:
        print(f"\n... (총 {len(pdf_result.content)}자 중 {preview_length}자 표시)")
else:
    print("추출된 콘텐츠가 없습니다.")

📄 추출된 마크다운 콘텐츠 (미리보기)
<figure>

Microsoft

</figure>


<figure>
</figure>


<!-- PageHeader="JANUARY 2025" -->

ADVANCE
SUSTAINABILITY


# Accelerating Sustainability with AI: Innovations for a Better Future


<figure>

O

</figure>


<!-- PageBreak -->


<table>
<tr>
<th>Foreword</th>
<th>Actions &amp; Innovations</th>
<th>AI Investments</th>
<th>Data &amp; Infrastructure</th>
<th>Energy &amp; Resources</th>
<th>Policy &amp; Governance</th>
<th>Workforce capacity</th>
<th>Looking ahead</th>
</tr>
<tr>
<td>CONTENTS</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td colspan="2">Foreword</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td>3</td>
</tr>
<tr>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
</tr>
<tr>
<td colspan="2">Actions and Innovations</td>
<td></td>
<td></td>
<td></td>
<td></td>
<td></td>
<td>6</td>
</tr>
<tr>
<td></td>
<td>1 Invest in AI</td>
<td>to accelerate</td>
<td>sustainability</td>
<t

## 5. PPTX 문서 분석 (Layout 모델)

PowerPoint 프레젠테이션 파일도 Layout 모델로 분석할 수 있습니다.

> ⚠️ **참고**: PPTX 파일은 OCR_HIGH_RESOLUTION 기능을 지원하지 않습니다.

In [10]:
def analyze_pptx_to_markdown(file_path: str) -> AnalyzeResult:
    """
    PPTX 파일을 분석하여 마크다운 형식으로 컨텐츠를 추출합니다.
    
    Args:
        file_path: PPTX 파일 경로
        
    Returns:
        AnalyzeResult: 분석 결과 객체
    """
    # 파일 존재 여부 확인
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"파일을 찾을 수 없습니다: {file_path}")
    
    print(f"📂 PPTX 분석 시작: {file_path}")
    print("-" * 60)
    
    # 로컬 파일을 바이너리로 읽기
    with open(file_path, "rb") as f:
        file_content = f.read()
    
    # Layout 모델로 PPTX 분석 시작
    # PPTX 파일은 특별한 content_type 사용
    poller = document_intelligence_client.begin_analyze_document(
        "prebuilt-layout",  # Layout 모델 사용
        body=file_content,
        content_type="application/vnd.openxmlformats-officedocument.presentationml.presentation",  # PPTX 컨텐츠 타입
        output_content_format=DocumentContentFormat.MARKDOWN  # 마크다운 형식으로 출력
    )
    
    # 분석 완료 대기 및 결과 반환
    result: AnalyzeResult = poller.result()
    
    print(f"✅ PPTX 분석 완료!")
    return result

In [11]:
# PPTX 분석 실행
pptx_result = analyze_pptx_to_markdown(pptx_file_path)

📂 PPTX 분석 시작: samples/Cognitive Searvices and Content Intelligence.pptx
------------------------------------------------------------
✅ PPTX 분석 완료!


In [12]:
# PPTX 분석 결과 요약 출력
print_analysis_summary(pptx_result, "PPTX")


📊 PPTX 분석 결과 요약
📄 총 페이지 수: 11
   - 페이지 1: None x None None
   - 페이지 2: None x None None
   - 페이지 3: None x None None
   - 페이지 4: None x None None
   - 페이지 5: None x None None
   - 페이지 6: None x None None
   - 페이지 7: None x None None
   - 페이지 8: None x None None
   - 페이지 9: None x None None
   - 페이지 10: None x None None
   - 페이지 11: None x None None

📝 단락 수: 135


In [13]:
# PPTX 추출된 마크다운 콘텐츠 미리보기 (처음 2000자)
print("📄 추출된 마크다운 콘텐츠 (미리보기)")
print("=" * 60)

if pptx_result.content:
    preview_length = 2000
    content_preview = pptx_result.content[:preview_length]
    print(content_preview)
    if len(pptx_result.content) > preview_length:
        print(f"\n... (총 {len(pptx_result.content)}자 중 {preview_length}자 표시)")
else:
    print("추출된 콘텐츠가 없습니다.")

📄 추출된 마크다운 콘텐츠 (미리보기)
# Cognitive Search and Augmentation Combining Microsoft Cognitive Services and Azure Search

Luis Cabrera

Principal Program Manager

Content and Search Intelligence

<!-- PageBreak -->

We learn and use technology.

Technology learns us

and discovers ways to be helpful.

Answering

Coordinating

Listening

Observing

Anticipating

<!-- PageBreak -->

Microsoft AI

Paul

AI SERVICES

Bot Framework

CUSTOM SERVICES

Azure Machine Learning

AI ON DATA

AI INFRASTRUCTURE

Cosmos DB

AI COMPUTE

SQL

DB

SQL

DW

Data Lake

Spark

DSVM

Batch AI

ACS

CPU, GPU, FPGA

AI TOOLS

VS Tools for Machine Learning

Azure ML Studio

CODING & MANAGEMENT TOOLS

Azure ML Workbench

DEEP LEARNING FRAMEWORKS

Cognitive Toolkit

TensorFlow

Caffe

Others (Scikit-learn, MXNet, Keras, Gluon…)

CONVERSATIONAL AI

TRAINED SERVICES

Cognitive Services

Edge

Others (Pycharm, Jupyter notebooks…)

<!-- PageBreak -->

Vision

From faces to feelings, allow

your apps to understand

images a

## 6. 분석 결과를 마크다운 파일로 저장

추출된 콘텐츠를 마크다운 파일로 저장하여 활용할 수 있습니다.

In [14]:
def save_result_to_markdown(result: AnalyzeResult, output_path: str):
    """
    분석 결과를 마크다운 파일로 저장합니다.
    
    Args:
        result: Document Intelligence 분석 결과
        output_path: 저장할 마크다운 파일 경로
    """
    # 출력 디렉토리 생성
    output_dir = os.path.dirname(output_path)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
    
    # 마크다운 파일로 저장
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(result.content if result.content else "")
    
    print(f"✅ 마크다운 파일 저장 완료: {output_path}")

# 출력 폴더 설정
output_dir = "output"

# PDF 결과 저장
pdf_output_path = os.path.join(output_dir, "pdf_extracted_content.md")
save_result_to_markdown(pdf_result, pdf_output_path)

# PPTX 결과 저장
pptx_output_path = os.path.join(output_dir, "pptx_extracted_content.md")
save_result_to_markdown(pptx_result, pptx_output_path)

print(f"\n📁 출력 파일 위치: {os.path.abspath(output_dir)}")

✅ 마크다운 파일 저장 완료: output/pdf_extracted_content.md
✅ 마크다운 파일 저장 완료: output/pptx_extracted_content.md

📁 출력 파일 위치: /Users/kichul/Documents/project/ai-search-with-document-intelligence/ai-search-with-document-intelligence/tutorials/output


## 7. 추가: 상세 분석 정보 확인

페이지별 상세 정보, 테이블 내용 등을 확인할 수 있습니다.

In [15]:
def print_table_details(result: AnalyzeResult, max_tables: int = 3):
    """
    분석 결과에서 테이블 상세 정보를 출력합니다.
    
    Args:
        result: Document Intelligence 분석 결과
        max_tables: 출력할 최대 테이블 수
    """
    if not result.tables:
        print("테이블이 발견되지 않았습니다.")
        return
    
    print(f"📋 테이블 상세 정보 (최대 {max_tables}개 표시)")
    print("=" * 60)
    
    for idx, table in enumerate(result.tables[:max_tables]):
        print(f"\n테이블 {idx + 1}: {table.row_count}행 x {table.column_count}열")
        
        # 테이블 위치 정보
        if table.bounding_regions:
            for region in table.bounding_regions:
                print(f"  📍 위치: 페이지 {region.page_number}")
        
        # 테이블 셀 내용 출력 (처음 몇 개만)
        print(f"  셀 내용 (처음 10개):")
        for cell in table.cells[:10]:
            content = cell.content.replace('\n', ' ')[:50]  # 줄바꿈 제거, 50자 제한
            print(f"    [{cell.row_index},{cell.column_index}]: {content}")
        
        if len(table.cells) > 10:
            print(f"    ... 외 {len(table.cells) - 10}개 셀")

# PDF 테이블 상세 정보 출력
print("📄 PDF 문서의 테이블 정보")
print_table_details(pdf_result)

📄 PDF 문서의 테이블 정보
📋 테이블 상세 정보 (최대 3개 표시)

테이블 1: 14행 x 8열
  📍 위치: 페이지 2
  셀 내용 (처음 10개):
    [0,0]: Foreword
    [0,1]: Actions & Innovations
    [0,2]: AI Investments
    [0,3]: Data & Infrastructure
    [0,4]: Energy & Resources
    [0,5]: Policy & Governance
    [0,6]: Workforce capacity
    [0,7]: Looking ahead
    [1,0]: CONTENTS
    [1,1]: 
    ... 외 100개 셀

테이블 2: 6행 x 3열
  📍 위치: 페이지 6
  셀 내용 (처음 10개):
    [0,0]: In this section
    [0,2]: 
    [1,0]: 1
    [1,1]: Invest in AI for sustainability
    [1,2]: 8
    [2,0]: 2
    [2,1]: Develop digital and data infrastructure for the in
    [2,2]: 16
    [3,0]: 3
    [3,1]: Minimize resource use, expand access to carbon-fre
    ... 외 7개 셀


## 8. 좌표 정보(Polygon)와 콘텐츠 출력

Document Intelligence는 각 요소(라인, 단어, 단락 등)의 위치 정보를 polygon(다각형 좌표)으로 제공합니다.
이 좌표 정보를 활용하여 문서에서 특정 영역을 하이라이트하거나 추출할 수 있습니다.

### Polygon 좌표 형식
- 좌표는 `[x1, y1, x2, y2, x3, y3, x4, y4]` 형식의 8개 값으로 구성
- 왼쪽 상단부터 시계방향으로 4개의 꼭지점 좌표
- 단위는 페이지의 `unit` 속성에 따라 inch 또는 pixel

In [16]:
def print_page_lines_with_polygon(result: AnalyzeResult, page_num: int = 1, max_lines: int = 10):
    """
    특정 페이지의 라인별 좌표 정보와 콘텐츠를 출력합니다.
    
    Args:
        result: Document Intelligence 분석 결과
        page_num: 출력할 페이지 번호 (1부터 시작)
        max_lines: 출력할 최대 라인 수
    """
    if not result.pages:
        print("페이지 정보가 없습니다.")
        return
    
    # 해당 페이지 찾기
    page = None
    for p in result.pages:
        if p.page_number == page_num:
            page = p
            break
    
    if not page:
        print(f"페이지 {page_num}을 찾을 수 없습니다.")
        return
    
    print(f"📄 페이지 {page_num} 라인별 좌표 및 콘텐츠")
    print(f"   페이지 크기: {page.width} x {page.height} {page.unit}")
    print("=" * 80)
    
    if not page.lines:
        print("라인 정보가 없습니다.")
        return
    
    for idx, line in enumerate(page.lines[:max_lines]):
        # 라인 콘텐츠
        content = line.content[:60] + "..." if len(line.content) > 60 else line.content
        
        # Polygon 좌표 (8개 값: 4개 꼭지점의 x, y)
        polygon = line.polygon if line.polygon else []
        
        print(f"\n라인 {idx + 1}:")
        print(f"  📝 콘텐츠: {content}")
        if polygon:
            # polygon은 [x1, y1, x2, y2, x3, y3, x4, y4] 형식
            print(f"  📍 좌표 (polygon):")
            print(f"      좌상단: ({polygon[0]}, {polygon[1]})")
            print(f"      우상단: ({polygon[2]}, {polygon[3]})")
            print(f"      우하단: ({polygon[4]}, {polygon[5]})")
            print(f"      좌하단: ({polygon[6]}, {polygon[7]})")
    
    if len(page.lines) > max_lines:
        print(f"\n... 외 {len(page.lines) - max_lines}개 라인")

# PDF 1페이지 라인 좌표 출력
print_page_lines_with_polygon(pdf_result, page_num=1, max_lines=5)

📄 페이지 1 라인별 좌표 및 콘텐츠
   페이지 크기: 8.5 x 11.0 LengthUnit.INCH

라인 1:
  📝 콘텐츠: Microsoft
  📍 좌표 (polygon):
      좌상단: (1.002, 0.6978)
      우상단: (2.1182, 0.692)
      우하단: (2.1195, 0.9501)
      좌하단: (1.0035, 0.9564)

라인 2:
  📝 콘텐츠: JANUARY 2025
  📍 좌표 (polygon):
      좌상단: (5.2523, 2.6058)
      우상단: (6.24, 2.6055)
      우하단: (6.2401, 2.7319)
      좌하단: (5.2523, 2.7321)

라인 3:
  📝 콘텐츠: ADVANCE
  📍 좌표 (polygon):
      좌상단: (6.6159, 2.5109)
      우상단: (7.4058, 2.509)
      우하단: (7.4061, 2.6549)
      좌하단: (6.6162, 2.6568)

라인 4:
  📝 콘텐츠: SUSTAINABILITY
  📍 좌표 (polygon):
      좌상단: (6.6146, 2.6769)
      우상단: (7.9584, 2.6764)
      우하단: (7.9584, 2.828)
      좌하단: (6.6147, 2.8285)

라인 5:
  📝 콘텐츠: Accelerating
  📍 좌표 (polygon):
      좌상단: (0.6255, 5.2703)
      우상단: (4.0785, 5.3138)
      우하단: (4.0663, 6.0159)
      좌하단: (0.6143, 5.9318)

... 외 4개 라인


In [17]:
def print_words_with_confidence(result: AnalyzeResult, page_num: int = 1, max_words: int = 15):
    """
    특정 페이지의 단어별 좌표, 콘텐츠, 신뢰도를 출력합니다.
    
    Args:
        result: Document Intelligence 분석 결과
        page_num: 출력할 페이지 번호 (1부터 시작)
        max_words: 출력할 최대 단어 수
    """
    if not result.pages:
        print("페이지 정보가 없습니다.")
        return
    
    # 해당 페이지 찾기
    page = None
    for p in result.pages:
        if p.page_number == page_num:
            page = p
            break
    
    if not page or not page.words:
        print(f"페이지 {page_num}의 단어 정보가 없습니다.")
        return
    
    print(f"\n📝 페이지 {page_num} 단어별 좌표, 콘텐츠 및 신뢰도")
    print("=" * 100)
    
    for idx, word in enumerate(page.words[:max_words]):
        content = word.content[:25] + ".." if len(word.content) > 25 else word.content
        confidence = f"{word.confidence:.2%}" if word.confidence else "N/A"
        polygon = word.polygon if word.polygon else []
        
        print(f"\n단어 {idx + 1}: '{content}' (신뢰도: {confidence})")
        if polygon and len(polygon) >= 8:
            print(f"  📍 전체 좌표 (polygon):")
            print(f"      좌상단: ({polygon[0]}, {polygon[1]})")
            print(f"      우상단: ({polygon[2]}, {polygon[3]})")
            print(f"      우하단: ({polygon[4]}, {polygon[5]})")
            print(f"      좌하단: ({polygon[6]}, {polygon[7]})")
    
    if len(page.words) > max_words:
        print(f"\n... 외 {len(page.words) - max_words}개 단어 (총 {len(page.words)}개)")

# PDF 1페이지 단어 정보 출력
print_words_with_confidence(pdf_result, page_num=1, max_words=10)


📝 페이지 1 단어별 좌표, 콘텐츠 및 신뢰도

단어 1: 'Microsoft' (신뢰도: 99.40%)
  📍 전체 좌표 (polygon):
      좌상단: (1.0028, 0.699)
      우상단: (2.1195, 0.6926)
      우하단: (2.1195, 0.9546)
      좌하단: (1.0035, 0.9577)

단어 2: 'JANUARY' (신뢰도: 99.50%)
  📍 전체 좌표 (polygon):
      좌상단: (5.2523, 2.6087)
      우상단: (5.8658, 2.6074)
      우하단: (5.8656, 2.7334)
      좌하단: (5.2526, 2.7342)

단어 3: '2025' (신뢰도: 99.10%)
  📍 전체 좌표 (polygon):
      좌상단: (5.93, 2.6069)
      우상단: (6.2422, 2.6066)
      우하단: (6.2422, 2.7315)
      좌하단: (5.9298, 2.7326)

단어 4: 'ADVANCE' (신뢰도: 99.50%)
  📍 전체 좌표 (polygon):
      좌상단: (6.6159, 2.5112)
      우상단: (7.4098, 2.5096)
      우하단: (7.4098, 2.6573)
      좌하단: (6.6164, 2.6573)

단어 5: 'SUSTAINABILITY' (신뢰도: 99.20%)
  📍 전체 좌표 (polygon):
      좌상단: (6.6148, 2.6791)
      우상단: (7.9632, 2.6768)
      우하단: (7.9632, 2.8282)
      좌하단: (6.6147, 2.8289)

단어 6: 'Accelerating' (신뢰도: 99.30%)
  📍 전체 좌표 (polygon):
      좌상단: (0.6243, 5.2777)
      우상단: (4.0785, 5.3175)
      우하단: (4.0785, 6.0176)
      좌하단

In [18]:
def print_paragraphs_with_polygon(result: AnalyzeResult, max_paragraphs: int = 5):
    """
    단락별 좌표 정보와 콘텐츠를 출력합니다.
    
    Args:
        result: Document Intelligence 분석 결과
        max_paragraphs: 출력할 최대 단락 수
    """
    if not result.paragraphs:
        print("단락 정보가 없습니다.")
        return
    
    print(f"\n📑 단락별 좌표 및 콘텐츠 (총 {len(result.paragraphs)}개)")
    print("=" * 100)
    
    for idx, paragraph in enumerate(result.paragraphs[:max_paragraphs]):
        # 단락 콘텐츠 (100자 제한)
        content = paragraph.content[:100] + "..." if len(paragraph.content) > 100 else paragraph.content
        content = content.replace('\n', ' ')  # 줄바꿈 제거
        
        # 단락 역할 (제목, 본문 등)
        role = paragraph.role if hasattr(paragraph, 'role') and paragraph.role else "본문"
        
        # Bounding Region 정보
        bounding_regions = paragraph.bounding_regions if paragraph.bounding_regions else []
        
        print(f"\n단락 {idx + 1} [{role}]:")
        print(f"  📝 콘텐츠: {content}")
        
        if bounding_regions:
            for region in bounding_regions:
                page_num = region.page_number
                polygon = region.polygon if region.polygon else []
                print(f"  📍 페이지: {page_num}")
                if polygon and len(polygon) >= 8:
                    print(f"     전체 좌표 (polygon):")
                    print(f"        좌상단: ({polygon[0]}, {polygon[1]})")
                    print(f"        우상단: ({polygon[2]}, {polygon[3]})")
                    print(f"        우하단: ({polygon[4]}, {polygon[5]})")
                    print(f"        좌하단: ({polygon[6]}, {polygon[7]})")
    
    if len(result.paragraphs) > max_paragraphs:
        print(f"\n... 외 {len(result.paragraphs) - max_paragraphs}개 단락")

# PDF 단락 좌표 정보 출력
print_paragraphs_with_polygon(pdf_result, max_paragraphs=5)


📑 단락별 좌표 및 콘텐츠 (총 878개)

단락 1 [본문]:
  📝 콘텐츠: Microsoft
  📍 페이지: 1
     전체 좌표 (polygon):
        좌상단: (1.002, 0.6978)
        우상단: (2.1182, 0.692)
        우하단: (2.1196, 0.9506)
        좌하단: (1.0034, 0.9564)

단락 2 [ParagraphRole.PAGE_HEADER]:
  📝 콘텐츠: JANUARY 2025
  📍 페이지: 1
     전체 좌표 (polygon):
        좌상단: (5.2523, 2.6058)
        우상단: (6.24, 2.6055)
        우하단: (6.2401, 2.7319)
        좌하단: (5.2523, 2.7321)

단락 3 [본문]:
  📝 콘텐츠: ADVANCE SUSTAINABILITY
  📍 페이지: 1
     전체 좌표 (polygon):
        좌상단: (6.6145, 2.5093)
        우상단: (7.9583, 2.5088)
        우하단: (7.9584, 2.828)
        좌하단: (6.6147, 2.8285)

단락 4 [ParagraphRole.TITLE]:
  📝 콘텐츠: Accelerating Sustainability with AI: Innovations for a Better Future
  📍 페이지: 1
     전체 좌표 (polygon):
        좌상단: (0.6142, 5.2703)
        우상단: (6.0628, 5.2696)
        우하단: (6.0631, 8.0764)
        좌하단: (0.6145, 8.077)

단락 5 [본문]:
  📝 콘텐츠: O
  📍 페이지: 1
     전체 좌표 (polygon):
        좌상단: (6.5971, 7.808)
        우상단: (6.7955, 7.8147)
        우하단: (6

In [19]:
def print_figures_with_polygon(result: AnalyzeResult):
    """
    그림/이미지의 좌표 정보와 캡션을 출력합니다.
    
    Args:
        result: Document Intelligence 분석 결과
    """
    if not result.figures:
        print("그림/이미지 정보가 없습니다.")
        return
    
    print(f"\n🖼️  그림/이미지 좌표 정보 (총 {len(result.figures)}개)")
    print("=" * 100)
    
    for idx, figure in enumerate(result.figures):
        # 캡션 정보
        caption = ""
        if hasattr(figure, 'caption') and figure.caption:
            caption = figure.caption.content[:80] + "..." if len(figure.caption.content) > 80 else figure.caption.content
        
        # Bounding Region 정보
        bounding_regions = figure.bounding_regions if figure.bounding_regions else []
        
        print(f"\n그림 {idx + 1}:")
        if caption:
            print(f"  📝 캡션: {caption}")
        
        if bounding_regions:
            for region in bounding_regions:
                page_num = region.page_number
                polygon = region.polygon if region.polygon else []
                print(f"  📍 페이지: {page_num}")
                if polygon and len(polygon) >= 8:
                    print(f"     전체 좌표 (polygon):")
                    print(f"        좌상단: ({polygon[0]}, {polygon[1]})")
                    print(f"        우상단: ({polygon[2]}, {polygon[3]})")
                    print(f"        우하단: ({polygon[4]}, {polygon[5]})")
                    print(f"        좌하단: ({polygon[6]}, {polygon[7]})")
                    # 바운딩 박스 크기 계산
                    min_x = min(polygon[0], polygon[6])
                    min_y = min(polygon[1], polygon[3])
                    max_x = max(polygon[2], polygon[4])
                    max_y = max(polygon[5], polygon[7])
                    width = max_x - min_x
                    height = max_y - min_y
                    print(f"     크기: {width} x {height}")

# PDF 그림 좌표 정보 출력
print_figures_with_polygon(pdf_result)


🖼️  그림/이미지 좌표 정보 (총 28개)

그림 1:
  📍 페이지: 1
     전체 좌표 (polygon):
        좌상단: (0.5814, 0.6407)
        우상단: (2.1169, 0.6407)
        우하단: (2.1172, 0.9906)
        좌하단: (0.5816, 0.9906)
     크기: 1.5358 x 0.3499

그림 2:
  📍 페이지: 1
     전체 좌표 (polygon):
        좌상단: (0.0, 1.4607)
        우상단: (4.717, 1.4695)
        우하단: (4.7097, 4.8647)
        좌하단: (0.0, 4.8559)
     크기: 4.717 x 3.404

그림 3:
  📍 페이지: 1
     전체 좌표 (polygon):
        좌상단: (5.7446, 6.919)
        우상단: (8.4242, 6.9207)
        우하단: (8.423, 10.2396)
        좌하단: (5.7435, 10.2376)
     크기: 2.6807000000000007 x 3.3205999999999998

그림 4:
  📍 페이지: 3
     전체 좌표 (polygon):
        좌상단: (4.2268, 3.0156)
        우상단: (8.4383, 3.0139)
        우하단: (8.4376, 9.6885)
        좌하단: (4.2264, 9.6897)
     크기: 4.2119 x 6.675800000000001

그림 5:
  📍 페이지: 4
     전체 좌표 (polygon):
        좌상단: (4.6624, 7.0681)
        우상단: (7.6986, 7.0682)
        우하단: (7.6987, 10.0155)
        좌하단: (4.6624, 10.0152)
     크기: 3.0362999999999998 x 2.94739999999999

## 9. 정리

이 튜토리얼에서 진행한 내용:

1. **Azure Document Intelligence 클라이언트 설정**: API 키 또는 `DefaultAzureCredential`을 사용한 인증
2. **Layout 모델 사용**: PDF, PPTX 등 다양한 문서 형식 분석
3. **마크다운 출력**: `DocumentContentFormat.MARKDOWN` 옵션으로 마크다운 형식 추출
4. **분석 결과 활용**: 페이지, 테이블, 단락, 그림 정보 추출
5. **좌표 정보 활용**: polygon 좌표로 문서 요소의 정확한 위치 파악